<a href="https://colab.research.google.com/github/ajit-gvs/HCR/blob/master/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from PIL import Image
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
directory='/content/data'
text=open('/content/words_new.txt')

samples,labels=data_label()
trainx,testx,trainy,testy=data_split(samples,labels):
test_accuracy,train_accuracy,parameters=model(trainx,trainy,testx,testy)


In [ ]:
def data_label():
  labels=[]
  samples=[]
 

 
  
  for filename in os.listdir(directory):
    image_orig=Image.open('/content/data/'+filename,'r')
    img=image_binarization(image_orig)
    
    
    lines=line_segmentation(img)
    
    for i in range(len(lines)):
      for line in text:
        
         
        lineSplit = line.strip().split(" ")
        linesplit=lineSplit[0].split("-")
        string= linesplit[0]+'-'+linesplit[1]
        if string==filename:
          count=count+1
    if count==len(lines): 
      for i in range(len(lines)):
        words=word_segmentation(lines[i])
        for line in text:
          lineSplit = line.strip().split(' ')
          filename_split=filename.split('.')
          if lineSplit[0]==filename_split[0]+'-'+str(0)+str(i):
            word_split = lineSplit[8].split('|')
            if len(word_split)==len(words):
              for j in range(len(words)):
                characters=char_segmentation(words[j])
                for k in range(len(characters)):
                  if len(word_split[k])==len(characters):
                    character=cv2.resize(characters[k],(28,28))
                    samples.append(character)
                    for l in word_split[k]:
                      labels.append(l)
    
    
                  
  print(len(samples))
  return  samples,labels





def data_split(samples,labels):
  samples = np.array(samples)
  labels = np.array(labels)   
  lb = LabelBinarizer()
  LABELS = lb.fit_transform(labels) 
  trainx,testx,trainy,testy = train_test_split(samples, LABELS, test_size=0.2, random_state=42)

  return trainx,testx,trainy,testy




def image_binarization(image):
  # converting image to grayscale
  image_grayscale=image.convert('L')
  img=np.array(image_grayscale)
   
  img[img <128] = 0
  img[img >=128] = 254 

  #making white as foreground pixels and black as background pixels
  img[img==0]=255
  img[img==254]=0
  
  return img


def line_segmentation(img):
  start_matrix=[]
  end_matrix=[]

  #matrix to get the start and end points of a line
  
  lines=[]
  begin_matrix=[]
  stop_matrix=[]
  del_start_matrix=[]
  del_end_matrix=[]

  horizontal_hist = np.sum(img,axis=1,keepdims=True)/255
  start_count=0

  for i in range(len(horizontal_hist)):

    if horizontal_hist[i]>0 and horizontal_hist[i-1]==0:
      start_count+=1
      start_matrix.append(i)

    if horizontal_hist[i]==0 and start_count>0 and horizontal_hist[i-1]>0:
      end_matrix.append(i)

  
  
  if len(start_matrix)==len(end_matrix):
    for i in range(len(start_matrix)):
      if end_matrix[i]-start_matrix[i]<20:
        del_start_matrix.append(i)
        del_end_matrix.append(i)
    for i in range(len(start_matrix)):
      count=0
      for j in range(len(del_start_matrix)):
        if i==del_start_matrix[j]:
          count=count+1
      if count==0:
        begin_matrix.append(start_matrix[i])
    for i in range(len(end_matrix)):
      count=0
      for j in range(len(del_end_matrix)):
        if i==del_end_matrix[j]:
          count=count+1
      if count==0:
        stop_matrix.append(end_matrix[i])
    
    for i in range(len(begin_matrix)):
      lines.append(img[begin_matrix[i]:stop_matrix[i],:])

      

      

  
  return lines




def word_segmentation(img):
  start_matrix=[]
  end_matrix=[]

  #matrix to get the start and end points of a word
  dissection_matrix=[]
  
  words=[]
  m,n=img.shape
  
  length=[]
  vertical_hist = np.sum(img,axis=0,keepdims=True)/255
  
  
  start_count=0
  
  
  for i in range(len(vertical_hist[0])):
    if vertical_hist[0][i]>0 and vertical_hist[0][i-1]==0:
      start_count+=1
      start_matrix.append(i)
    if vertical_hist[0][i]==0 and start_count>0 and vertical_hist[0][i-1]>0:
      end_matrix.append(i)
  
  
  length_mag=0
  for i in range(len(start_matrix)):
    if i>0:
      length_mag=(start_matrix[i]-end_matrix[i-1])
      length.append(length_mag)

  max=np.max(length)
     
  avg=max/3

  dissection_matrix.append([start_matrix[0],end_matrix[0]])
  j=0
  for i in range(len(length)-1):
    
    if length[i]> avg:
      dissection_matrix.append([start_matrix[i+1],end_matrix[i+1]])
      j=j+1
      
    if length[i]<=avg:
      dissection_matrix[j][1]=end_matrix[i+1]
  for i in range(len(dissection_matrix)):
    words.append(img[0:m,dissection_matrix[i][0]:dissection_matrix[i][1]])
  
  return words


def char_segmentation(img):
  start_matrix=[]
  dissection_matrix=[]
  delete_matrix=[]
  address_matrix=[]
  
  
  characters=[]
  m,n=img.shape

  vertical_hist = np.sum(img,axis=0,keepdims=True)/255
  start_matrix.append(0)

  for i in range(len(vertical_hist[0])):
    if vertical_hist[0][i]<10:
      start_matrix.append(i)

  for i in range(len(start_matrix)-1):
    if start_matrix[i+1]-start_matrix[i]<10:
      delete_matrix.append(i)
  
  for i in range(len(start_matrix)):
    count=0
    for j in range(len(delete_matrix)):
      if  i==delete_matrix[j]:
        count=count+1
    if count==0:
      address_matrix.append(start_matrix[i])


  for i in range(len(address_matrix)-1):
    dissection_matrix.append([address_matrix[i],address_matrix[i+1]])
  for i in range(len(dissection_matrix)):
    characters.append(img[0:m,dissection_matrix[i][0]:dissection_matrix[i][1]])
    
    
  return characters






def buildCNN(trainx,parameters):
  
   
  W1 = parameters['W1']
  W2 = parameters['W2']
  W3 = parameters['W3']
  W4 = parameters['W4']
  W5 = parameters['W5']
  #Convolution Layer1
  C1 = tf.nn.conv2d(trainx,W1, strides = [1,1,1,1], padding = 'SAME')
  #Activation Layer1
  A1 = tf.nn.relu(C1)
  #Batch Normalization Layer1
  #BN1 = tf.nn.batch_normalization(A1)
  #Max Pooling Layer1
  P1 = tf.nn.max_pool(BN1, ksize = [1,3,3,1], strides = [1,3,3,1], padding = 'VALID')


  
  #Convolution Layer2
  C2 = tf.nn.conv2d(P1,W2, strides = [1,1,1,1], padding = 'SAME')
  #Activation Layer2
  A2 = tf.nn.relu(C2)
  #Batch Normalization Layer2
  #BN2 = tf..batch_normalization(A2)
  #Max Pooling Layer2
  P2 = tf.nn.max_pool(BN2, ksize = [1,3,3,1], strides = [1,3,3,1], padding = 'VALID')


 
  #Convolution Layer3
  C3 = tf.nn.conv2d(P2,W3, strides = [1,1,1,1], padding = 'SAME')
  #Activation Layer3
  A3 = tf.nn.relu(C3)
  #Batch Normalization Layer2
  #BN3 = tf.batch_normalization(A3)
  #Max Pooling Layer3
  P3 = tf.nn.max_pool(BN3, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')


    #Convolution Layer4
  C4 = tf.nn.conv2d(P3,W4, strides = [1,1,1,1], padding = 'SAME')
  #Activation Layer4
  A4 = tf.nn.relu(C4)
  #Batch Normalization Layer4
  #BN4 = tf.batch_normalization(A4)
  #Max Pooling Layer4
  P4 = tf.nn.max_pool(BN4, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')


    #Convolution Layer5
  C5 = tf.nn.conv2d(P4,W5, strides = [1,1,1,1], padding = 'SAME')
  #Activation Layer5
  A5 = tf.nn.relu(C5)
  #Batch Normalization Layer2
  #BN5 = tf.batch_normalization(A5)
  #Max Pooling Layer2
  A5 = tf.nn.max_pool(BN5, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')



  # FLATTEN
  Flatten = tf.compat.v1.layers.flatten(P3)
  Dense1 = tf.compat.v1.layers.dense(Flatten, units, activation=None)
  Batch_norm1 = tf.nn.batch_normalization(Dense1)


  output = tf.compat.v1.layers.dense(Batch_norm1, CLASSES, activation_fn=None)
  return output
  

  


  

def initialize_parameters():
    
    W1 = tf.get_variable("W1", [3, 3, 1, 32], initializer = tf.contrib.layers.xavier_initializer())
    W2 = tf.get_variable("W2", [3, 3, 32, 64], initializer = tf.contrib.layers.xavier_initializer())
    W3 = tf.get_variable("W3", [2, 2, 64, 128], initializer = tf.contrib.layers.xavier_initializer())
    W4 = tf.get_variable("W4", [2, 2, 128, 256], initializer = tf.contrib.layers.xavier_initializer())
    W5 = tf.get_variable("W5", [2, 2, 256, 512], initializer = tf.contrib.layers.xavier_initializer())

    parameters = {"W1": W1,
                  "W2": W2,
                  "W3": W3,
                  "W4": W4,
                  "W5": W5,
                  }
   
    return parameters


def compute_cost(output, Y):

    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = output, labels = Y))
    
    return cost


def random_batches(trainx, trainy, batch_size = 64):
    # number of training examples
    no_samples =  len(trainx)               
    batches = []
    
    # Shuffle (X, Y)
    permutation = list(np.random.permutation(no_samples))
    shuffled_X = trainx[permutation,:,:]
    shuffled_Y = trainy[permutation,:]

    # Partition (shuffled_X, shuffled_Y) and remove the end case.
    num_complete_batches = math.floor(no_samples/batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for i in range(0, num_complete_batches):
        batch_X = shuffled_X[i * batch_size : i * batch_size + batch_size,:,:,:]
        batch_Y = shuffled_Y[i * batch_size : i * batch_size + batch_size,:]
        batch = (batch_X, batch_Y)
        batches.append(batch)
    
    # Handling the end case if last mini-batch < batch_size
    if no_samples % batch_size != 0:
        batch_X = shuffled_X[num_complete_batches * batch_size : no_samples,:,:,:]
        batch_Y = shuffled_Y[num_complete_batches * batch_size : no_samples,:]
        batch = (batch_X, batch_Y)
        batches.append(batch)
    
    return mini_batches

def model(trainx, trainy, testx, testy, learning_rate = 0.001,
          num_epochs = 100, minibatch_size = 64):

    
    
    
    costs = []        
     # Initializing parameters
    parameters = initialize_parameters()                                
    
    

     # Forward propagation
    output = buildCNN(trainx,parameters)
    
    # Cost function
    cost = compute_cost(output, trainy)
    
    # Backpropagation
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    # Initializing all the variables globally
    init = tf.global_variables_initializer()
     
    # Starting the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        
        sess.run(init)
        
        
        for epoch in range(num_epochs):

            batch_cost = 0.
            # number of batches of size batch_size in the train set
            num_batches = int(no_samples / batch_size)
            batches = random_batches(X_train, Y_train,batch_size)

            for batch in batches:

                # Select a batch
                (batch_X, batch_Y) = batch
                
                _ , temp_cost = sess.run([optimizer, cost], feed_dict={trainx: batch_X, trainy: batch_Y})
                
                batch_cost += temp_cost / num_batches
                

            
            if epoch % 5 == 0:
                print ("Cost after epoch %i: %f" % (epoch, batch_cost))
            if epoch % 1 == 0:
                costs.append(batch_cost)
        # plot the cost
        #plt.plot(np.squeeze(costs))
        #plt.ylabel('cost')
        #plt.xlabel('iterations (per tens)')
        #plt.title("Learning rate =" + str(learning_rate))
        #plt.show()
        
        # lets save the parameters in a variable
        parameters = sess.run(parameters)
        print ("Parameters have been trained!")     
        
        # Calculate the correct predictions
        predict_op = tf.argmax(output, 1)
        correct_prediction = tf.equal(predict_op, tf.argmax(testy, 1))
        
        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        train_accuracy = accuracy.eval({X: trainx, Y: trainy})
        test_accuracy = accuracy.eval({X: testx, Y: testy})
        print("Train Accuracy:", train_accuracy)
        print("Test Accuracy:", test_accuracy)
        
        # Saving our trained model
        saver = tf.train.Saver()
        tf.add_to_collection('predict_op', predict_op)
        saver.save(sess, './my-CNN-model')        
                
        
      
              
                
    return train_accuracy, test_accuracy, parameters
